In [24]:
endereco_modelo = "http://192.168.0.230:1244"

In [25]:
import requests

requests.get(f"{endereco_modelo}/v1/models").json()

{'data': [{'id': 'qwen/qwen2.5-vl-7b',
   'object': 'model',
   'owned_by': 'organization_owner'},
  {'id': 'google/gemma-3n-e4b',
   'object': 'model',
   'owned_by': 'organization_owner'},
  {'id': 'text-embedding-nomic-embed-text-v1.5',
   'object': 'model',
   'owned_by': 'organization_owner'}],
 'object': 'list'}

In [26]:
import base64
import io
import requests
from pdf2image import convert_from_path

def extracao_de_pdf(endereco):
    # URL da API do LM Studio
    api_url = f"{endereco_modelo}/v1/chat/completions"  # Corrigido o endpoint

    # Caminho para o arquivo PDF que será processado
    pdf_file = endereco  # substitua pelo caminho do seu PDF

    # Converte cada página do PDF em uma imagem (ajuste o dpi se necessário)
    pages = convert_from_path(pdf_file, dpi=300)

    final_markdown = ""

    # Processa cada página do PDF
    for i, page in enumerate(pages):
        # Converte a imagem para PNG e codifica em base64
        buffered = io.BytesIO()
        page.save(buffered, format="PNG")
        img_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")
        
        # Cria o prompt para a API, incluindo a imagem em base64
        image_content = f"<image>{img_base64}</image>"
        
        # Monta o payload para a requisição à API do LM Studio
        payload = {
            "model": "qwen/qwen2.5-vl-7b",
            "messages": [
                {
                    "role": "system", 
                    "content": "You are an assistant specialized in OCR and Markdown formatting."
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": "Please perform OCR on this image and extract all text exactly as it appears, converting any tables or visual elements into Markdown format. Do not include any additional commentary or repetition."
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{img_base64}"
                            }
                        }
                    ]
                }
            ],
            "temperature": 0.0
        }
        
        # Adiciona headers necessários
        headers = {
            "Content-Type": "application/json"
        }
        
        # Envia a requisição POST para o LM Studio
        response = requests.post(api_url, json=payload, headers=headers)
        
        if response.status_code == 200:
            result = response.json()
            print(result)
            # Extrai o conteúdo da resposta
            page_markdown = result["choices"][0]["message"]["content"]
        else:
            page_markdown = f"Error: {response.status_code} - {response.text}"
        
        # Adiciona a resposta da página ao resultado final
        final_markdown += f"\n\n## Page {i+1}\n\n" + page_markdown
    return final_markdown


In [27]:
import os
import argparse

def encontrar_pdfs(diretorio):
    """
    Retorna uma lista com os caminhos completos de todos os arquivos PDF encontrados no diretório e subdiretórios.
    
    Args:
        diretorio (str): Caminho do diretório raiz para a busca
    
    Returns:
        list: Lista de strings com caminhos absolutos dos PDFs
    """
    caminhos_pdf = []
    
    for pasta_raiz, _, arquivos in os.walk(diretorio):
        for arquivo in arquivos:
            if arquivo.lower().endswith('.pdf'):
                caminho_completo = os.path.join(pasta_raiz, arquivo)
                caminhos_pdf.append(caminho_completo)
    
    return caminhos_pdf

In [28]:
diretorio_de_busca = "./data"

In [29]:
lista_pdfs = encontrar_pdfs(diretorio_de_busca)

In [30]:
lista_pdfs

['./data/tuberculosis-annual-epidemiological-report-2021.pdf',
 './data/zika-virus-disease-annual-epidemiological-report-2021.pdf']

In [31]:
data = []

In [32]:
import json
def export_to_json(data, json_path):
    with open(json_path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

In [33]:
for i in lista_pdfs:
    print(i)
    extracao = extracao_de_pdf(i)
    aux = {"page_content" : extracao, "metadata": {"source": i.split('/')[-1]}}
    data.append(aux)
    print(aux)
    json_path = 'dados.json'

    # Exportar para JSON
    export_to_json(data, json_path)

    print(f"Dados exportados para {json_path}")

./data/tuberculosis-annual-epidemiological-report-2021.pdf
{'id': 'chatcmpl-onazaa3zmmdsgw2xnd59ie', 'object': 'chat.completion', 'created': 1756094293, 'model': 'qwen/qwen2.5-vl-7b', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'SURVEILLANCE REPORT\n\nTuberculosis\nAnnual Epidemiological Report for 2021\n\n\nKey facts\n* For 2021, the 35 countries in the European Union/European Economic Area (EU/EEA) reported a total of 33,172 tuberculosis (TB) cases (7.4 per 100,000 population).\n* The EU/EEA is not on track to reach the goal of ending the TB epidemic by 2030.\n* The figures presented in this report should be interpreted with caution, given the potential impact of measures put in place to mitigate the COVID-19 pandemic across the EU/EEA.\n\n\nIntroduction\nTuberculosis (TB) is an infectious disease caused by a group of Mycobacterium species called the *Mycobacterium tuberculosis complex*. TB can affect any organ (extra-pulmonary tuberculosis). It is transmitte

In [ ]:
json_path = 'Racional_AWS_DC_2.json'

# Exportar para JSON
export_to_json(data, json_path)

print(f"Dados exportados para {json_path}")